# Importar bibliotecas generales

In [5]:
import numpy as np
import pandas as pd
import re as regex
import math as math

# Importar bibliotecas propias 

In [6]:
#biblioteca para completar valores Nan de la columna rooms
%run "fill_column_rooms.ipynb"

# Leer el data set desde el archivo y generar el data frame

In [7]:
pathArchivoDataSet = 'properatti.csv'
df = pd.read_csv(pathArchivoDataSet)

# Detalle generales del data frame

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [9]:
df.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [10]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

# La primer columna no tiene un nombre asignado

In [11]:
type(df.iloc[0,0])

numpy.int64

In [12]:
len(df.loc[df.index == df['Unnamed: 0']])

121220

## Para todas las filas del data frame, el valor de la primer columna es igual al valor del index. Se puede asumir que dicha columna corresponde a un campo ID. Se renombra la primer columna

In [13]:
df.rename(columns={'Unnamed: 0': 'Id'}, inplace = True)
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

# Limpieza del data set 

### Se eliminan columnas innecesarias

In [14]:
#quitamos la columna floor, ya que es difícil de inferir a partir de las demás columnas, por simplicidad se quita de los datos
df.drop(labels='floor', axis = 1, inplace = True)

In [15]:
#quitamos la columna image_thumbnail ya que no aporta información relevante
df = df.drop(labels='image_thumbnail', axis= 1)

In [16]:
#quitamos la columna properati_url ya que no aporta información relevante
df = df.drop(labels='properati_url', axis= 1)

In [17]:
#chequeamos las columnas que nos quedan
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'expenses', 'description', 'title'],
      dtype='object')

### Se eliminan datos (filas) que no pudieron ser completados

In [18]:
#se eliminan filas cuyo campo description está vacío, dicha columna se tomó como fuente para otras columnas
#al venir vacía otros datos no pueden ser completados
emptyDescriptionIndexes = df[df['description'].isnull()].index

#borramos las filas con los correspondientes indices
df.drop(emptyDescriptionIndexes , inplace=True)

#reseteamos los índices para poder seguir usándolos sin problemas
df.reset_index();

# Completar valores faltantes

### A partir del dato geonames_id, generamos tres columnas nuevas, que corresponden a la coordenada representativa de cada zona (representada por un geonames_id). Esta coordenada no corresponde a la de la propiedad

##### Leemos directamente el archivo CSV generado en la notebook auxiliar "Completar coordenadas desde geonames_id" . Para ver el proceso remitirse a dicha notebook

In [19]:
latLngCSVFileName = 'latLngFromGeonames.csv'
#cargamos el archivo
latLongDF = pd.read_csv(latLngCSVFileName)

In [20]:
#mergeamos el data frame leído con el da properatti usando el dato geonames_id
df = df.merge(latLongDF, how='left', left_on='geonames_id', right_on='geonames_id', suffixes=('', '_geonames'))

### Datos de coordenadas de propiedades

#### Armamos 2 DataFrame:
    (A) 1 DataFrame para Ordenada Latitud: de columnas 'place_name' y 'lat'
    (B) 1 DataFrame para Ordenada Longitud: de columnas 'place_name' y 'lon'

(A) DF de Ordenada Latitud

In [21]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 

dflatitud=df[['place_name', 'lat']]
dflatitud.head()

,place_name,lat
0,Mataderos,-34.661824
1,La Plata,-34.903883
2,Mataderos,-34.652262
3,Liniers,-34.647797
4,Centro,-38.002626


Comprobamos cantidad de filas completas de la columna 'lat' en comparación al total de filas

In [22]:
dflatitud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121218 entries, 0 to 121217
Data columns (total 2 columns):
place_name    121195 non-null object
lat           69669 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.8+ MB


De (A) nos quedamos solo con los campos con datos, de la columna 'lat', desechando los valores 'NaN'

In [23]:
dflatitud_1=dflatitud.dropna()
dflatitud_1.head()

,place_name,lat
0,Mataderos,-34.661824
1,La Plata,-34.903883
2,Mataderos,-34.652262
3,Liniers,-34.647797
4,Centro,-38.002626


Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lat' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [24]:
dflatitud_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69646 entries, 0 to 121215
Data columns (total 2 columns):
place_name    69646 non-null object
lat           69646 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


Calulamos la media de las latitudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Latitudes

In [25]:
dflatitud_1=dflatitud_1.groupby(['place_name']).mean()
dflatitud_1.head()

,lat
place_name,
Country Maschwitz Club,-34.377707
los alamos,-34.423989
9 de Abril,-34.756258
Abasto,-34.639208
Abril Club de Campo,-34.814314


Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [26]:
len(dflatitud_1)

959

(B) DF de Ordenada Longitud

###### Armo un subset sólo con las columnas 'place_name' y 'lon'

In [27]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 
dflongitud=df[['place_name', 'lon']]
dflongitud.head()

,place_name,lon
0,Mataderos,-58.508839
1,La Plata,-57.964330
2,Mataderos,-58.522982
3,Liniers,-58.516424
4,Centro,-57.549447


Comprobamos cantidad de filas completas de la columna 'lon' en comparación al total de filas

In [28]:
dflongitud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121218 entries, 0 to 121217
Data columns (total 2 columns):
place_name    121195 non-null object
lon           69669 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.8+ MB


De (B) nos quedamos solo con los campos con datos, de la columna 'lon', desechando los valores 'NaN'

In [29]:
dflongitud_1=dflongitud.dropna()
dflongitud_1.head()

,place_name,lon
0,Mataderos,-58.508839
1,La Plata,-57.964330
2,Mataderos,-58.522982
3,Liniers,-58.516424
4,Centro,-57.549447


Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lon' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [30]:
dflongitud_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69646 entries, 0 to 121215
Data columns (total 2 columns):
place_name    69646 non-null object
lon           69646 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


Calulamos la media de las longtudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Longitudes

In [31]:
dflongitud_1=dflongitud_1.groupby(['place_name']).mean()
dflongitud_1.head()

,lon
place_name,
Country Maschwitz Club,-58.755411
los alamos,-58.601876
9 de Abril,-58.498288
Abasto,-58.380827
Abril Club de Campo,-58.156066


Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [32]:
len(dflongitud_1)

959

Definimos un nuevo dataframe que una las columnas lat y lon de los puntos (A) y (B).
Para ello vamos utilizar el método ".merge()" de Pandas

In [33]:
dflatlon=pd.merge(dflatitud_1, dflongitud_1, left_on='place_name', right_on='place_name')
dflatlon

,lat,lon
place_name,,
Country Maschwitz Club,-34.377707,-58.755411
los alamos,-34.423989,-58.601876
9 de Abril,-34.756258,-58.498288
Abasto,-34.639208,-58.380827
Abril Club de Campo,-34.814314,-58.156066
...,...,...
William Morris,-34.581910,-58.655338
Yerba Buena,-26.813039,-65.294461
Zelaya,-35.053791,-59.660925


Tomamos al nuevo dataframe y generamos un nuevo índice para que 'place_name' vuelva a ser una variable del df

In [34]:
dflatlon.reset_index(inplace=True)
dflatlon

,place_name,lat,lon
0,Country Maschwitz Club,-34.377707,-58.755411
1,los alamos,-34.423989,-58.601876
2,9 de Abril,-34.756258,-58.498288
3,Abasto,-34.639208,-58.380827
4,Abril Club de Campo,-34.814314,-58.156066
...,...,...,...
954,William Morris,-34.581910,-58.655338
955,Yerba Buena,-26.813039,-65.294461
956,Zelaya,-35.053791,-59.660925
957,Zárate,-34.096436,-59.067763


Tomamos al dataframe original (df) y hacemos un Join con Pandas, sumando al final del DF las columnas con las medias de 'lat' y 'lon'

In [35]:
df = df.merge(dflatlon, how='left', left_on='place_name', right_on='place_name', suffixes=('','_mediaPorZona'))
df.head(5)

,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,rooms,expenses,description,title,lat-lon_geonames,lat_geonames,lon_geonames,lat_mediaPorZona,lon_mediaPorZona
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,1550.000000,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,"-34.66667,-58.5",-34.666670,-58.500000,-34.666184,-58.518986
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,"-35,-58",-35.000000,-58.000000,-34.924338,-57.966073
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,1309.090909,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,"-34.66667,-58.5",-34.666670,-58.500000,-34.666184,-58.518986
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,"-34.64286,-58.52386",-34.642860,-58.523860,-34.643470,-58.516909
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,1828.571429,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,"-38.0026256,-57.5494468",-38.002626,-57.549447,-38.000247,-57.547260


### Datos de localidades: place_with_parent_names, place_name, country_name, state_name

In [36]:
df[['place_with_parent_names', 'place_name','country_name','state_name']].isnull().sum()

place_with_parent_names     0
place_name                 23
country_name                0
state_name                  0
dtype: int64

##### Solo faltan 23 valores en la columna 'place_name' (información de barrio, zona, ciudad, etc.) que se intentarán obtener del la columna 'place_with_parent_names'.

In [37]:
#seteamos la longitud del output para mejor lectura
pd.set_option('display.max_colwidth', -1)

##### Tomamos las columnas necesarias para completar los faltantes

In [38]:
df[['place_with_parent_names', 'place_name','country_name','state_name']]

,place_with_parent_names,place_name,country_name,state_name
0,|Argentina|Capital Federal|Mataderos|,Mataderos,Argentina,Capital Federal
1,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,La Plata,Argentina,Bs.As. G.B.A. Zona Sur
2,|Argentina|Capital Federal|Mataderos|,Mataderos,Argentina,Capital Federal
3,|Argentina|Capital Federal|Liniers|,Liniers,Argentina,Capital Federal
4,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Centro,Argentina,Buenos Aires Costa Atlántica
...,...,...,...,...
121213,|Argentina|Capital Federal|Belgrano|,Belgrano,Argentina,Capital Federal
121214,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|Beccar|,Beccar,Argentina,Bs.As. G.B.A. Zona Norte
121215,|Argentina|Capital Federal|Villa Urquiza|,Villa Urquiza,Argentina,Capital Federal
121216,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Plaza Colón|,Plaza Colón,Argentina,Buenos Aires Costa Atlántica


##### Se crea una nueva columna con los valores de la columna 'place_with_parent_names' en forma de lista

In [39]:
def disgrega(valor):
    return valor.strip('|').split('|')

df['lista_auxiliar'] = df['place_with_parent_names'].apply(lambda x: disgrega(x))

In [40]:
df['lista_auxiliar']

0         [Argentina, Capital Federal, Mataderos]                              
1         [Argentina, Bs.As. G.B.A. Zona Sur, La Plata]                        
2         [Argentina, Capital Federal, Mataderos]                              
3         [Argentina, Capital Federal, Liniers]                                
4         [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Centro]     
                                        ...                                    
121213    [Argentina, Capital Federal, Belgrano]                               
121214    [Argentina, Bs.As. G.B.A. Zona Norte, San Isidro, Beccar]            
121215    [Argentina, Capital Federal, Villa Urquiza]                          
121216    [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Plaza Colón]
121217    [Argentina, Capital Federal]                                         
Name: lista_auxiliar, Length: 121218, dtype: object

##### Se calcula cuantos elementos tiene cada lista

In [41]:
df['conteo'] = df['lista_auxiliar'].apply(lambda x: len(x))
df['conteo']

0         3
1         3
2         3
3         3
4         4
         ..
121213    3
121214    4
121215    3
121216    4
121217    2
Name: conteo, Length: 121218, dtype: int64

##### Se determina cual es el mayor y menor números de elemntos en un registro de 'place_with_parent_names'

In [42]:
df['conteo'].max()

5

In [43]:
df['conteo'].min()

2

##### Se necesitarán 5 columnas como máximo para desempacar las distintas jerarquías de las locaciones. Mediante el comando apply se obtienen 5 columnas extras con información de localización por regiones.

In [44]:
def chequeo2(valor):
    if len(valor) > 2:
        return valor[2]
    
    
def chequeo3(valor):
    if len(valor) > 3:
        return valor[3]
    
def chequeo4(valor):
    if len(valor) > 4:
        return valor[4]

df['loc1'] = df['lista_auxiliar'].apply(lambda x: x[0])
df['loc2'] = df['lista_auxiliar'].apply(lambda x: x[1])
df['loc3'] = df['lista_auxiliar'].apply(lambda x: chequeo2(x))
df['loc4'] = df['lista_auxiliar'].apply(lambda x: chequeo3(x))
df['loc5'] = df['lista_auxiliar'].apply(lambda x: chequeo4(x))

In [45]:
daux = df[(df.loc5.notnull())]
daux[['loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

,loc1,loc2,loc3,loc4,loc5
17,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio El Golf
33,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio El Golf
278,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Las Glorietas
416,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Los Lagos
446,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Las Glorietas
...,...,...,...,...,...
117009,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,QBay Yacht
118208,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Los Lagos
118839,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Barrancas del Lago
119581,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Barrancas del Lago


##### Verificamos que los registros que tenían place_name vacíos tengan valores válidos en las nuevas columnas para cubrir el faltante

In [46]:
len(df[df['place_name'].isna() & ~df['loc5'].isna()])

0

In [47]:
df[df['place_name'].isna()][['lat','lon','geonames_id','place_name','place_with_parent_names', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

,lat,lon,geonames_id,place_name,place_with_parent_names,loc1,loc2,loc3,loc4,loc5
6489,-34.400897,-58.638098,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
10201,-34.400704,-58.638067,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
11450,-34.402688,-58.626261,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
14838,-34.400799,-58.638174,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
18621,-34.403278,-58.626350,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
21921,-34.399704,-58.638405,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
23663,-34.400516,-58.641800,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
24721,-34.400729,-58.637778,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
38855,-34.425087,-58.579659,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
45969,-34.396074,-58.637380,NaN,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None


##### Property_Type => OK

In [48]:
#Vemos los tipos de propiedades existentes, y su correspondiente cantidad

df['property_type'].value_counts()

apartment    71063
house        40268
PH           5751 
store        4136 
Name: property_type, dtype: int64

In [49]:
#Revisamos si alguna linea tiene el tipo de propiedad vacio
#Vemos que no, que todas las 121.220 lineas existentes tienen el campo completo

df['property_type'].value_counts().sum()

121218

In [50]:
house = df.loc[:, 'property_type'] == 'house'
print("La cantidad de house es: ",house.sum())

apartment = df.loc[:, 'property_type'] == 'apartment'
print("La cantidad de apartment es: ",apartment.sum())

PH = df.loc[:, 'property_type'] == 'PH'
print("La cantidad de PH es: ",PH.sum())

store = df.loc[:, 'property_type'] == 'store'
print("La cantidad de store es: ",store.sum())

tipodepropiedad = house & apartment & PH & store
print("La cantidad de propiedades sin tipo es: ",tipodepropiedad.sum())

La cantidad de house es:  40268
La cantidad de apartment es:  71063
La cantidad de PH es:  5751
La cantidad de store es:  4136
La cantidad de propiedades sin tipo es:  0


##### Surface (covered y total)

###### Opcion 1: funcion para obtener metros cuadrados (a partir de surface_total_in_m2 y surface_covered_in_m2)

In [51]:
print('Cantidad registros surface_covered_in_m2 sin valor: ' + str(df['surface_covered_in_m2'].isnull().sum()))
print('Cantidad registros surface_total_in_m2 sin valor: ' + str(df['surface_total_in_m2'].isnull().sum()))

Cantidad registros surface_covered_in_m2 sin valor: 19906
Cantidad registros surface_total_in_m2 sin valor: 39326


###### Vamos a tratar de llenar los datos faltantes de superficie total en el dataset, rellenando con el promedio por barrio de cada publicacion

In [52]:
#Primero vamos a ordenar el dataset para poder tener los partidos en orden y limpios
df.rename(index=str, inplace=True)

In [53]:
#Primero hacmemos data wrangling de la columna "place_with_parent_names"

df.place_with_parent_names = df.place_with_parent_names.map(str.lower) #llevo a minusculas para evitar duplicados

grouped_places = df.groupby(['place_with_parent_names']) #agrupo por place_with_parent_names

dictio_places = grouped_places.groups.keys() #genero diccionario de places

cantidad_places = len(dictio_places) #cuento la cantidad de places distintos

print("Cantidad de place_with_parent_names distintos: ",cantidad_places)

Cantidad de place_with_parent_names distintos:  1164


In [54]:
count_x_places = grouped_places.agg({"Id": "count"}) #agrupo y cuento

count_x_places = count_x_places.rename(index=str, columns={"Id": "cantidad"}) #renombro la columna por cantidad

count_x_places = count_x_places.sort_values(by=['cantidad'], ascending=False) #ordeno por cantidad descendente

In [55]:
len(count_x_places.query("cantidad > 50")) # places con mas de <n> registros. Solo consulta

261

In [56]:
list_places = [sub_places.split('|') for sub_places in count_x_places.index]

df_places = pd.DataFrame(list_places, 
                         index = count_x_places.index, 
                         columns =['none1','pais','provincia','partido','localidad','barrio','none2'])

df_places = df_places.drop(['none1', 'none2'], axis=1) # elimino none1 y none2
df_places = df_places.drop(['pais'], axis=1) # elimino pais ya que todos son argentina
df_places[df_places.barrio.notnull()] # el unico que tiene barrios es tigre-nordelta
df_places = df_places.drop(['barrio'], axis=1) # elimino barrio tambien

In [57]:
# Para cada columna busco vacios y asigno None

for column_name in df_places.columns:
    df_places[column_name][df_places[column_name].apply(lambda column_name: True if regex.search('^\s*$', str(column_name)) else False)]=None

df_places = df_places.sort_values(by=['provincia','partido','localidad']) #ordeno

# creo df de provincias partidos y localidades
df_provincias = pd.DataFrame(df_places.provincia.unique(),columns=['nombre'])
df_partidos = pd.DataFrame(df_places.partido.unique(),columns=['nombre'])
df_localidades = pd.DataFrame(df_places.localidad.unique(),columns=['nombre'])

def buscar_reemplazar_place_column(row,column_name,df_base):
    if row[column_name]:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_places.provincia =  df_places.apply(buscar_reemplazar_place_column,args=('provincia',df_provincias),axis=1)
df_places.partido =  df_places.apply(buscar_reemplazar_place_column,args=('partido',df_partidos),axis=1)
df_places.localidad =  df_places.apply(buscar_reemplazar_place_column,args=('localidad',df_localidades),axis=1)
df_places.sample(5)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  app.launch_new_instance()


,provincia,partido,localidad
place_with_parent_names,,,
|argentina|córdoba|mina clavero|,10,267.0,NaN
"|argentina|bs.as. g.b.a. zona norte|tigre|barrio cerrado ""isla del sol""|",0,8.0,192.0
|argentina|bs.as. g.b.a. zona norte|general san martín|barrio parque general san martin|,0,1.0,44.0
|argentina|capital federal|boedo|,5,136.0,NaN
|argentina|buenos aires interior|tandil|tandil|,4,121.0,557.0


In [58]:
# agrega la relacion para las columnas que se vayan pasando respecto al dataframe de provincias, localidades y partidos

def agregar_columna_place(row,column_name,test):
    if (row.place_with_parent_names):
        return df_places.loc[row.place_with_parent_names][column_name]
    
provincias = df.apply(agregar_columna_place,args=('provincia','random'),axis=1)
df['provincia'] = pd.Series(provincias, index=df.index)
partidos = df.apply(agregar_columna_place,args=('partido','random'),axis=1)
df['partido'] = pd.Series(partidos, index=df.index)
localidades = df.apply(agregar_columna_place,args=('localidad','random'),axis=1)
df['localidad'] = pd.Series(localidades, index=df.index)
df.sample(3)

,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,lista_auxiliar,conteo,loc1,loc2,loc3,loc4,loc5,provincia,partido,localidad
62216,62217,sell,apartment,Haedo,|argentina|bs.as. g.b.a. zona oeste|morón|haedo|,Argentina,Bs.As. G.B.A. Zona Oeste,NaN,NaN,NaN,...,"[Argentina, Bs.As. G.B.A. Zona Oeste, Morón, Haedo]",4,Argentina,Bs.As. G.B.A. Zona Oeste,Morón,Haedo,None,1.0,18.0,307.0
20383,20384,sell,apartment,Córdoba,|argentina|córdoba|córdoba|,Argentina,Córdoba,3860259.0,"-31.4106876,-64.1979202",-31.410688,...,"[Argentina, Córdoba, Córdoba]",3,Argentina,Córdoba,Córdoba,None,None,10.0,234.0,NaN
109915,109917,sell,house,Villa General Belgrano,|argentina|córdoba|villa general belgrano|,Argentina,Córdoba,3832734.0,"-31.9643768457,-64.5490398348",-31.964377,...,"[Argentina, Córdoba, Villa General Belgrano]",3,Argentina,Córdoba,Villa General Belgrano,None,None,10.0,311.0,NaN


In [59]:
df['state_name'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25559
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12068
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste        9322 
Buenos Aires Interior           2291 
Río Negro                       808  
Neuquén                         733  
Mendoza                         681  
Tucumán                         674  
Corrientes                      583  
Misiones                        464  
Entre Ríos                      369  
Salta                           278  
Chubut                          259  
San Luis                        252  
La Pampa                        157  
Formosa                         65   
Chaco                           57   
San Juan                        40   
Tierra Del Fuego                31   
Catamarca                       27   
Jujuy                           26   
Santa Cruz                      20   
La Rioja    

In [60]:
df['provincia'].value_counts()

5.0     32316
0.0     25559
2.0     13952
10.0    12068
24.0    10172
3.0     10006
1.0     9322 
4.0     2291 
19.0    808  
18.0    733  
16.0    681  
27.0    674  
9.0     583  
17.0    464  
11.0    369  
20.0    278  
8.0     259  
22.0    252  
14.0    157  
12.0    65   
7.0     57   
21.0    40   
26.0    31   
6.0     27   
13.0    26   
23.0    20   
25.0    4    
15.0    4    
Name: provincia, dtype: int64

In [61]:
# Analisis del cambio de columnas realizado con place_with parent names

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)
df_place_name = df.place_name

df_place_name_not_in = df_place_name[~(df_place_name.isin(df_partidos.nombre))]
df_place_name_not_in_loc = df_place_name_not_in[~(df_place_name_not_in.isin(df_localidades.nombre))]
df_place_name_not_in_loc_prov = df_place_name_not_in_loc[~(df_place_name_not_in_loc.isin(df_provincias.nombre))]
df_place_name_not_in_loc_prov.unique()
#Puedo eliminar la columna place_name

array(['barrio el golf', 'barrio las glorietas', 'barrio los lagos',
       'enyoi', 'qbay yacht', 'barrio la alameda', 'islas del canal',
       'barrio los alisos', 'barrioportezuelo', nan, 'barrio la isla',
       'barrio los tilos', 'barrio los sauces', 'barrio cabos del lago',
       'barrio los castores', 'barrio barrancas del lago',
       'barrio el yacht'], dtype=object)

In [62]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en partidos

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)

def buscar_reemplazar_place_definidos(row,column_name,df_base): # funcion que buscar y reemplaza de la columna base (provincia, localidad, partido)
    a = df_base.nombre.str.contains(row[column_name], regex=False).any()
    if a:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_place_name = df.place_name
mask_in_partidos = (df_place_name.isin(df_partidos.nombre))
mask_not_column_partido = df.partido.isnull()

print("Partidos con null:" + str(df.partido.isnull().sum()))

Partidos con null:4780


In [63]:
df_reemplazar_part = df[mask_not_column_partido&mask_in_partidos].apply(buscar_reemplazar_place_definidos,args=('place_name',df_partidos),axis=1)
df.partido.update(df_reemplazar_part)
print("Partidos con null luego de procesar:" + str(df.partido.isnull().sum()))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  if __name__ == '__main__':


Partidos con null luego de procesar:1904


In [64]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en localidades

df_place_name = df.place_name
mask_in_localidades = (df_place_name.isin(df_localidades.nombre))
mask_not_column_localidad = df.localidad.isnull()

print("Localidades con null: " + str(df.localidad.isna().sum()))

df_reemplazar_loc = df[mask_not_column_localidad&mask_in_localidades].apply(buscar_reemplazar_place_definidos,args=('place_name',df_localidades),axis=1)
df.localidad.update(df_reemplazar_loc)

print("Localidades con null luego de procesar: " + str(df.localidad.isna().sum()))

Localidades con null: 80801


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  if __name__ == '__main__':


Localidades con null luego de procesar: 68991


###### Ahora vamos a limpiar la columna surface

In [65]:
df_title = df[df.title.notnull()]
df_title.title = df_title.title.map(str.lower)
pattern_m2 = regex.compile("(\d+\s*) m2")

def get_m2(row):
    result = pattern_m2.search(row)
    try:
        str_aux = result.group(1)
        array_m2 = str_aux.split()
        m2 = array_m2[-1]
        try:
            m2 = float(m2)
            return m2
        except:
            return np.nan;
    except:
        return np.nan

print("Cantidad de nulos m2 total surface: ",df.surface_total_in_m2.isnull().sum())

m2_from_title = df_title.title.apply(get_m2)
m2_from_title[m2_from_title.notnull()]
df.surface_total_in_m2.update(m2_from_title)

print("Cantidad de nulos luego de procesar: ",df.surface_total_in_m2.isnull().sum())

Cantidad de nulos m2 total surface:  39326
Cantidad de nulos luego de procesar:  38871


In [66]:
#Vamos a generar los 39.328 datos faltantes en surface_total en la tabla, y lo vamos a hacer
#rellenando con el promedio del barrio de cada propiedad

print("Cantidad de nulos en surface_total_in_m2 antes:",df.surface_total_in_m2.isnull().sum())

# creamos una proporción de metros cubiertos sobre metros totales - no puede ser mayor a 1!
propcubierto = df.surface_covered_in_m2 / df.surface_total_in_m2
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

Cantidad de nulos en surface_total_in_m2 antes: 38871


In [67]:
#Relevamos algunos casos anómalos, donde la proporción da mayor a uno (inverosímil). Desestimamos estos 1200 registros para este procedimiento.
tempmask = propcubierto > 1
np.sum(tempmask)

1200

In [68]:
#Creamos una nueva variable sin esos datos anómalos
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

In [69]:
#Hay dos casos anómalos donde superficie cubierta es cero, los reemplazamos por np.nan para evitar conflictos en el siguiente paso.
masksurface0 = (df.surface_covered_in_m2 == 0)
df.surface_covered_in_m2[masksurface0].fillna(np.nan)

#Agregamos la columna al dataframe
df['propcubierto']=propcubierto_clean

#Agrupamos por provincia(partido) el porcentaje promedio de m2cubierto/m2total
avg_propcubiertobarrio = df.groupby('partido')["propcubierto"].mean().sort_values(ascending = False)

#Cantidad de datos para calcular la proporcion 
avg_propcubiertobarriocount = df.groupby('partido')["propcubierto"].count().sort_values(ascending = False)

#Condición de la regla 1
removerporcantidadmask = avg_propcubiertobarriocount > 30

#Cantidad de datos existentes en tabla
datos_en_tabla = df.groupby('partido')['partido'].count().sort_values(ascending = False)

#Divido los datos existentes sobre los datos totales y obtengo la relación para la regla DOS
proporcion = avg_propcubiertobarriocount / datos_en_tabla
proporcion.round(2).sort_values(ascending = True)

#Condición de la regla 2
removerporproporcionmask = proporcion > 0.25

#Genero máscara con ambas condiciones
proporcionmask2 = removerporcantidadmask & removerporproporcionmask
propvalidados = avg_propcubiertobarrio[proporcionmask2]

#Ahora iteramos por las propiedades que tienen el dato faltante de superficie total y les inputamos la proporción promedio
#del barrio al que pertenecen, usando como dato la superficie cubierta.
# SUPERFICIE TOTAL = (SUPERFICIE CUBIERTA / PROPORCION CUBIERTO TOTAL)

surface_total_in_m2_clean = []
for index, row in df.iterrows():
    
    if pd.isnull(row.surface_total_in_m2):    
        if(row.partido in propvalidados.index):        
            surface_total_in_m2_clean.append(row.surface_covered_in_m2 / propvalidados.loc[row.partido]) #VERSION CON PROPVALIDADOS NO CORRE
        else:
            surface_total_in_m2_clean.append(row.surface_total_in_m2)    
    else:
        surface_total_in_m2_clean.append(row.surface_total_in_m2)
df["surface_total_in_m2_nueva"] = surface_total_in_m2_clean
df["surface_total_in_m2_nueva"]

#dropeamos los casos NaN
surfacetotalnueva_condatos = df.surface_total_in_m2_nueva.dropna()

#y finalmente, los reemplazamos en la columna "surface_total_in_m2_nueva" original del DF
df.surface_total_in_m2.update(surfacetotalnueva_condatos)

print("Cantidad de nulos en surface_total_in_m2 despues:",df.surface_total_in_m2.isnull().sum())

Cantidad de nulos en surface_total_in_m2 despues: 21180


In [70]:
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'expenses', 'description', 'title', 'lat-lon_geonames', 'lat_geonames',
       'lon_geonames', 'lat_mediaPorZona', 'lon_mediaPorZona',
       'lista_auxiliar', 'conteo', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5',
       'provincia', 'partido', 'localidad', 'propcubierto',
       'surface_total_in_m2_nueva'],
      dtype='object')

In [71]:
df.surface_total_in_m2.isnull().sum()

21180

##### Completar el valor del campo rooms

In [72]:
def completarValoresFaltantesEnFila(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms utilizando el método definido en la notebbok auxiliar fill_column_rooms (entrega 1).ipynb
    updatedDataFameRow = updateRoomsFromRowData(dataFrameRow)
        
    return updatedDataFameRow

In [73]:
df = df.apply(completarValoresFaltantesEnFila, axis=1);

###### Datos en las columnas description y title

In [74]:
print('Cantidad de registros description con valor nulo: ' + str(df.description.isna().sum()))
print('Cantidad de registros title con valor nulo: ' + str(df.title.isna().sum()))

Cantidad de registros description con valor nulo: 0
Cantidad de registros title con valor nulo: 0


###### Columna price_usd_per_m2

In [75]:
#definimos un diccionario con valores de conversion entre monedas locales y el USD
conversion_USD_a_monedas_locales = { 'ARS': 63, 'PEN': 3.53, 'UYU': 43.41, 'USD': 1}

In [76]:
#funcion que recibe una fila del data frame y un diccionario con las conversiones entre monedas locales y el USD
#este metodo intenta calcular el precio por m2 en USD a partir de otras columnas del data frame
def updatePriceUSDPerM2(dataFrameRow, dolarConversion):
    
    conversion = 1
    
    #si el dato currency no tiene valor, asumimos que los valores son en USD (conversion = 1)
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_usd_per_m2)):
        #precio por m2 USD= precio total USD / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
        
        #precio por m2 USD = (precio total $ * conversion moneda) / superficie total        
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price * conversion) / dataFrameRow.surface_total_in_m2
    
        #precio por m2 USD = precio por m2 * conversion moneda
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_per_m2)):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_per_m2 * conversion
            
        #precio por m2 USD = precio aprox por m2 USD / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
            
        #precio por m2 USD = precio aprox por m2 local currency * conversion moenda / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price_aprox_local_currency * conversion) / dataFrameRow.surface_total_in_m2
            
    return dataFrameRow

In [77]:
#contamos price per m2 en USD antes de la ejecucion
df['price_usd_per_m2'].isna().sum()

52601

In [78]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePriceUSDPerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [79]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_usd_per_m2'].isna().sum()

28248

###### Columna price_per_m2

In [80]:
#similar al calculo del m2 por USD, el calculo del m2 en moneda local usa otras columnas del data frame para obtener el valor
#recibe tambien un diccionario con la conversion de monedas locales a USD
def updatePricePerM2(dataFrameRow, dolarConversion):
            
    #si el dato currency no tiene valor, asumimos que el valor precio por m2 a guardar es en ARS
    conversion = 1 / dolarConversion['ARS'] #si el dolar esta a 65 ARS => el factor de conversión de USD a ARS es 1 / 65
    
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = 1 / dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_per_m2)):
        #precio por m2 = precio aprox total local currency / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_local_currency / dataFrameRow.surface_total_in_m2
        
        #precio por m2 = precio por m2 USD / conversion moneda
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_usd_per_m2)):
            dataFrameRow.price_per_m2 = dataFrameRow.price_usd_per_m2 / conversion
            
        #precio por m2 USD = (precio aprox por m2 USD / conversion) / superficie total
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_per_m2 = (dataFrameRow.price_aprox_usd / conversion) / dataFrameRow.surface_total_in_m2
             
    return dataFrameRow

In [81]:
#contamos price per m2 en USD antes de la ejecucion
df['price_per_m2'].isna().sum()

33561

In [ ]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePricePerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [ ]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_per_m2'].isna().sum()